In [56]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
from torch import long


In [57]:
df = pd.read_csv(r'C:\Users\HP\Desktop\datasets\unik\ml_unik.csv')
df = df.copy()
df

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M
...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,5,Blue,O
236,30839,834042.000000,1194.0000,-10.63,5,Blue,O
237,8829,537493.000000,1423.0000,-10.73,5,White,A
238,9235,404940.000000,1112.0000,-11.23,5,White,A


In [58]:
df = pd.get_dummies(df,dtype=float)

In [59]:
X = df.drop(['Star type'], axis=1)
y = df['Star type']
X = X.to_numpy()
y = y.to_numpy()

In [60]:
from sklearn.preprocessing  import StandardScaler
sc = StandardScaler()
X_std = sc.fit_transform(X)

In [61]:
from sklearn.model_selection import train_test_split
X_train_std, X_test_std, y_train, y_test = train_test_split(X_std, y, train_size=0.8, test_size=0.2, random_state=0)

In [62]:
X_train_std = torch.as_tensor(X_train_std, dtype=float)
X_test_std = torch.as_tensor(X_test_std, dtype=float)
y_train = torch.as_tensor(y_train, dtype=long)
y_test = torch.as_tensor(y_test, dtype=long)

In [63]:
class net(nn.Module):
    def __init__(self, hidden_n):
        super(net, self).__init__()
        self.fc1 = nn.Linear(30, hidden_n)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_n, hidden_n)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_n, 6)
        self.double()

    def forward(self, x):
        x = self.fc1(x) 
        x = self.act1(x) 
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x) 
        return x  


In [64]:
Net = net(30)
n_epoch = 500
crit = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Net.parameters(), lr = 0.01)


In [65]:
for epoch in range(n_epoch):
    optimizer.zero_grad()
    outputs = Net(X_train_std)
    loss = crit(outputs, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(loss.item())

1.7935540765884446
0.005628336900651818
0.0014962059073830494
0.0006689661716090916
0.000372852193036631


In [66]:
with torch.no_grad():
    outputs = Net(X_test_std)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(accuracy)

1.0
